In [1]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

sys.path.append("/global/homes/r/rtojeiro/SV/python")
import VI_utils as VI

import matplotlib.pyplot as plt 

pd.set_option('display.max_rows', 5)

In [2]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/'
tiles = ['68002']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
#obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 

In [3]:
VI_dir = os.environ['HOME']+'/SV/VI_files/SV0/LRG/'
save_filename = 'truth_table_LRG_v1.1.csv'

### Read individual VI files and concatenate all "VI Issue" fields by TargetID

In [4]:
vi = VI.read_individual_VI(VI_dir)

Reading all individual VI files in /global/homes/r/rtojeiro/SV/VI_files/SV0/LRG/
desi-vi_SV0_LRG_tile68002_night20200315_28_SAE.csv
desi-vi_SV0_LRG_tile68002_night20200315_27_MAR.csv
desi-vi_SV0_LRG_tile68002_night20200315_11_ACE.csv
desi-vi_SV0_LRG_tile68002_night20200315_21_PNT.csv
desi-vi_SV0_LRG_tile68002_night20200315_8_SAE.csv
desi-vi_SV0_LRG_tile68002_night20200315_5_SAE.csv
desi-vi_SV0_LRG_tile68002_night20200315_4_ACE.csv
desi-vi_SV0_LRG_tile68002_night20200315_19_RTR.csv
desi-vi_SV0_LRG_tile68002_night20200315_19_ACE.csv
desi-vi_SV0_LRG_tile68002_night20200315_3_ace.csv
desi-vi_SV0_LRG_tile68002_night20200315_17_PNT.csv
desi-vi_SV0_LRG_tile68002_night20200315_9_SAE.csv
desi-vi_SV0_LRG_tile68002_night20200315_4_unknown-user.csv
desi-vi_SV0_LRG_tile68002_night20200315_21_SAE.csv
desi-vi_SV0_LRG_tile68002_night20200315_18_HSH.csv
desi-vi_SV0_LRG_tile68002_night20200315_22_MAR.csv
desi-vi_SV0_LRG_tile68002_night20200315_14_SAE.csv
desi-vi_SV0_LRG_tile68002_night20200315_13_KSD.cs

In [5]:
vi

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment
0,35185971871876878,-1,0.33.0.dev3890,0,GALAXY,0.9038,PNT,3,--,0.9038,GALAXY,--
1,35185971871877391,-1,0.33.0.dev3890,0,GALAXY,0.4024,PNT,3,--,0.4024,GALAXY,--
...,...,...,...,...,...,...,...,...,...,...,...,...
4408,35185983813062463,-1,0.33.0.dev3890,0,GALAXY,0.1475,ACE,4,--,--,--,--
4409,35185983817252984,-1,0.33.0.dev3890,0,GALAXY,0.5461,ACE,4,--,--,--,--


In [6]:
#create new column - concatenates all entries in the "VI issue" field
vi['all VI issue'] = vi.groupby('TargetID')['VI issue'].transform(lambda x: '|'.join(x))

In [7]:
vi

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,all VI issue
0,35185971871876878,-1,0.33.0.dev3890,0,GALAXY,0.9038,PNT,3,--,0.9038,GALAXY,--,--|--|--
1,35185971871877391,-1,0.33.0.dev3890,0,GALAXY,0.4024,PNT,3,--,0.4024,GALAXY,--,--|--|--
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4408,35185983813062463,-1,0.33.0.dev3890,0,GALAXY,0.1475,ACE,4,--,--,--,--,--|--|--
4409,35185983817252984,-1,0.33.0.dev3890,0,GALAXY,0.5461,ACE,4,--,--,--,--,--|--|--


### Make truth table

Read all the partial truth table files and concatenate into one

In [8]:
filenames = ['truth_table_LRGs_sets1_21_28.txt','truth_table_LRGs_sets2_10.txt', 'truth_table_LRGs_sets11_20.txt']
#filenames = ['truth_table_LRGs_sets2_10.txt', 'truth_table_LRGs_sets11_20.txt']

truth_table = pd.read_csv(VI_dir+filenames[0])
for i in range(1,len(filenames)):
    partial_table = pd.read_csv(VI_dir+filenames[i])
    truth_table = truth_table.append(partial_table, ignore_index=True,sort=False)


In [9]:
truth_table

,TargetID,Redrock z,Redrock spectype,best redshift,best spectype,vi_combined_flag,N_VI,all VI comments,merger comment,DELTACHI2,FIBER,FLUX_G,FLUX_R,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z
0,35185923930984962,0.7168,GALAXY,0.7168,GALAXY,3.714286,7,--|--| several emission and absorption feature...,none,425.232418,515,2.183917,5.244880,15.983506,0.601037,1.443447,4.398831
1,35185923930985294,0.8047,GALAXY,0.8047,GALAXY,2.857143,7,Noisy spectra|--|h&k|--|--|--|--,"flag disagreement, take mean.",99.264655,516,0.068111,0.503776,3.286969,0.042047,0.310991,2.029113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408,35185971884462441,0.8292,GALAXY,0.8292,GALAXY,3.000000,3,--|--|--,"flag disagreement, take mean.",196.821232,3939,0.042960,0.492728,2.893630,0.029453,0.337814,1.983875
1409,35185971884462443,0.8564,GALAXY,0.8564,GALAXY,4.000000,3,--|--|--,none,243.757560,3600,0.241223,0.676728,3.536092,0.153386,0.430309,2.248483


### Add the 'all VI issue' field to the truth table

In [10]:
vi[['TargetID', 'all VI issue']]

,TargetID,all VI issue
0,35185971871876878,--|--|--
1,35185971871877391,--|--|--
...,...,...
4408,35185983813062463,--|--|--
4409,35185983817252984,--|--|--


In [11]:
truth_table=truth_table.merge(vi[['TargetID', 'all VI issue']], on='TargetID', how='left')

In [12]:
truth_table

,TargetID,Redrock z,Redrock spectype,best redshift,best spectype,vi_combined_flag,N_VI,all VI comments,merger comment,DELTACHI2,FIBER,FLUX_G,FLUX_R,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,all VI issue
0,35185923930984962,0.7168,GALAXY,0.7168,GALAXY,3.714286,7,--|--| several emission and absorption feature...,none,425.232418,515,2.183917,5.244880,15.983506,0.601037,1.443447,4.398831,--|--|--|--|--|--|--
1,35185923930984962,0.7168,GALAXY,0.7168,GALAXY,3.714286,7,--|--| several emission and absorption feature...,none,425.232418,515,2.183917,5.244880,15.983506,0.601037,1.443447,4.398831,--|--|--|--|--|--|--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4557,35185971884462443,0.8564,GALAXY,0.8564,GALAXY,4.000000,3,--|--|--,none,243.757560,3600,0.241223,0.676728,3.536092,0.153386,0.430309,2.248483,--|--|--
4558,35185971884462443,0.8564,GALAXY,0.8564,GALAXY,4.000000,3,--|--|--,none,243.757560,3600,0.241223,0.676728,3.536092,0.153386,0.430309,2.248483,--|--|--


### Add more information from the FIBERMAP and ZBEST files

In [13]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

tspec_df = tspec['TARGETID','ZWARN', 'ZERR' ].to_pandas()
tf_df = tf['TARGETID','MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z'].to_pandas()

tf_df = tf_df.rename(columns={"TARGETID": "TargetID"})
tspec_df = tspec_df.rename(columns={"TARGETID": "TargetID"})

truth_table = truth_table.merge(tf_df, how='left', on='TargetID')
truth_table = truth_table.merge(tspec_df, how='left', on='TargetID')

In [14]:
truth_table.keys()

Index(['TargetID', 'Redrock z', 'Redrock spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments',
       'merger comment', 'DELTACHI2', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z',
       'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'all VI issue',
       'MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z', 'ZWARN',
       'ZERR'],
      dtype='object')

### Rename columns

* TargetID -> TARGETID
* best redshift -> best z
* vi_combined_flag -> best flag


In [15]:
truth_table = truth_table.rename(columns={"TargetID":"TARGETID", "best redshift":"best z", "vi_combined_flag":"best flag"})

In [16]:
truth_table.keys()

Index(['TARGETID', 'Redrock z', 'Redrock spectype', 'best z', 'best spectype',
       'best flag', 'N_VI', 'all VI comments', 'merger comment', 'DELTACHI2',
       'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R',
       'FIBERFLUX_Z', 'all VI issue', 'MW_TRANSMISSION_G', 'MW_TRANSMISSION_R',
       'MW_TRANSMISSION_Z', 'ZWARN', 'ZERR'],
      dtype='object')

In [17]:
truth_table.to_csv(VI_dir+save_filename)